In [ ]:
# If running in Colab, uncomment:
!pip install shapely plotly

import math
from dataclasses import dataclass

import numpy as np
from shapely.geometry import Polygon
from shapely.ops import unary_union
import plotly.graph_objects as go

In [ ]:
# === USER INPUT: SITE GEOMETRY ===
# Define your site boundary as a list of (x, y) coordinates in meters.
# Must be a closed polygon (first and last point will be closed automatically).

site_coords = [
    (0.0, 0.0),
    (40.0, 0.0),
    (40.0, 25.0),
    (0.0, 25.0),
    # no need to repeat first point; Polygon will close it
]

site_polygon = Polygon(site_coords)

if not site_polygon.is_valid:
    raise ValueError("Site polygon is not valid. Check coordinates.")

site_area = site_polygon.area
site_bounds = site_polygon.bounds

print(f"Site area: {site_area:.2f} m²")
print(f"Bounds (minx, miny, maxx, maxy): {site_bounds}")

Site area: 1000.00 m²
Bounds (minx, miny, maxx, maxy): (0.0, 0.0, 40.0, 25.0)


In [ ]:
@dataclass
class ZoningConfig:
    front_setback: float  # meters
    rear_setback: float   # meters
    side_setback: float   # meters
    max_far: float        # floor area ratio
    max_height: float     # meters
    floor_to_floor: float # meters per storey
    max_coverage: float   # 0–1, fraction of site area (optional, use 1.0 if not constrained)

# === USER INPUT: ZONING RULES ===
zoning = ZoningConfig(
    front_setback=3.0,
    rear_setback=3.0,
    side_setback=3.0,
    max_far=3.0,
    max_height=24.0,
    floor_to_floor=3.0,
    max_coverage=1.0,
)

zoning

ZoningConfig(front_setback=3.0, rear_setback=3.0, side_setback=3.0, max_far=3.0, max_height=24.0, floor_to_floor=3.0, max_coverage=1.0)

In [ ]:
def apply_uniform_setback(polygon: Polygon, setback: float) -> Polygon:
    """
    Applies a uniform inward setback using a negative buffer.
    Positive 'setback' shrinks the polygon.
    """
    if setback <= 0:
        return polygon
    shrunk = polygon.buffer(-setback)
    if shrunk.is_empty:
        raise ValueError("Setbacks too large; buildable area vanished.")
    # If result is multi-part, take union and largest piece
    if shrunk.geom_type == "MultiPolygon":
        shrunk = max(shrunk.geoms, key=lambda g: g.area)
    return shrunk

def apply_directional_setbacks(polygon: Polygon, zoning: ZoningConfig) -> Polygon:
    """
    Simplified: uses the maximum of front/rear/side as a uniform setback.
    For a more precise directional approach, you'd need orientation + street edge logic.
    """
    max_setback = max(zoning.front_setback, zoning.rear_setback, zoning.side_setback)
    return apply_uniform_setback(polygon, max_setback)

buildable_polygon = apply_directional_setbacks(site_polygon, zoning)
buildable_area = buildable_polygon.area

print(f"Buildable footprint area: {buildable_area:.2f} m²")
print(f"Coverage ratio: {buildable_area / site_area:.2f}")

Buildable footprint area: 646.00 m²
Coverage ratio: 0.65


In [ ]:
# Max GFA from FAR
max_gfa_far = zoning.max_far * site_area

# Max storeys from height
max_storeys_height = math.floor(zoning.max_height / zoning.floor_to_floor)

# Max storeys from FAR (if fully using buildable footprint each floor)
max_storeys_far = math.floor(max_gfa_far / buildable_area)

max_storeys = min(max_storeys_height, max_storeys_far)

print(f"Max GFA (FAR-based): {max_gfa_far:.2f} m²")
print(f"Max storeys (height-based): {max_storeys_height}")
print(f"Max storeys (FAR-based): {max_storeys_far}")
print(f"Controlling max storeys: {max_storeys}")

Max GFA (FAR-based): 3000.00 m²
Max storeys (height-based): 8
Max storeys (FAR-based): 4
Controlling max storeys: 4


In [ ]:
def polygon_to_prism_mesh(polygon: Polygon, height: float, z0: float = 0.0):
    """
    Convert a 2D shapely Polygon into a triangular prism mesh (vertices, faces).
    Returns:
        vertices: (N, 3) array
        faces: (M, 3) array of vertex indices (triangles)
    """
    exterior = np.array(polygon.exterior.coords)
    # Remove duplicate last point if present
    if np.allclose(exterior[0], exterior[-1]):
        exterior = exterior[:-1]

    n = len(exterior)
    # Bottom and top vertices
    bottom = np.column_stack([exterior[:, 0], exterior[:, 1], np.full(n, z0)])
    top = np.column_stack([exterior[:, 0], exterior[:, 1], np.full(n, z0 + height)])

    vertices = np.vstack([bottom, top])

    faces = []

    # Side faces (quads split into two triangles)
    for i in range(n):
        i_next = (i + 1) % n
        # bottom indices
        b0 = i
        b1 = i_next
        # top indices
        t0 = i + n
        t1 = i_next + n

        # Triangle 1: b0, b1, t1
        faces.append([b0, b1, t1])
        # Triangle 2: b0, t1, t0
        faces.append([b0, t1, t0])

    # Top face triangulation (fan)
    for i in range(1, n - 1):
        faces.append([n, n + i, n + i + 1])  # all top vertices offset by n

    # Bottom face triangulation (fan)
    for i in range(1, n - 1):
        faces.append([0, i + 1, i])

    return np.array(vertices), np.array(faces)

In [ ]:
class MassingVariant:
    def __init__(self, name, polygon, height, storeys, color):
        self.name = name
        self.polygon = polygon
        self.height = height
        self.storeys = storeys
        self.color = color
        self.vertices, self.faces = polygon_to_prism_mesh(polygon, height)
        self.gfa = polygon.area * storeys

    def summary(self):
        return {
            "name": self.name,
            "height": self.height,
            "storeys": self.storeys,
            "footprint_area": self.polygon.area,
            "gfa": self.gfa,
        }

variants = []

# Variant A: Maxed-out block (all storeys on full buildable footprint)
height_A = max_storeys * zoning.floor_to_floor
variant_A = MassingVariant(
    name="Max Block",
    polygon=buildable_polygon,
    height=height_A,
    storeys=max_storeys,
    color="royalblue",
)
variants.append(variant_A)

# Variant B: Stepped massing (e.g. terrace after half height)
step_storeys = max(1, max_storeys // 2)
height_B1 = step_storeys * zoning.floor_to_floor
height_B2 = (max_storeys - step_storeys) * zoning.floor_to_floor

# Inner footprint for upper portion (extra setback)
extra_stepback = 2.0  # meters
upper_polygon = apply_uniform_setback(buildable_polygon, extra_stepback)

variant_B_podium = MassingVariant(
    name="Stepped Podium",
    polygon=buildable_polygon,
    height=height_B1,
    storeys=step_storeys,
    color="seagreen",
)
variant_B_tower = MassingVariant(
    name="Stepped Upper",
    polygon=upper_polygon,
    height=height_B2,
    storeys=max_storeys - step_storeys,
    color="lightseagreen",
)

variants.extend([variant_B_podium, variant_B_tower])

# Print summaries
for v in variants:
    print(v.summary())

{'name': 'Max Block', 'height': 12.0, 'storeys': 4, 'footprint_area': 646.0, 'gfa': 2584.0}
{'name': 'Stepped Podium', 'height': 6.0, 'storeys': 2, 'footprint_area': 646.0, 'gfa': 1292.0}
{'name': 'Stepped Upper', 'height': 6.0, 'storeys': 2, 'footprint_area': 450.0, 'gfa': 900.0}


In [ ]:
def plot_massing_3d(site_polygon: Polygon, variants, show_axes=False):
    fig = go.Figure()

    # Plot site boundary (as a low transparent prism)
    site_vertices, site_faces = polygon_to_prism_mesh(site_polygon, height=0.1)
    fig.add_trace(
        go.Mesh3d(
            x=site_vertices[:, 0],
            y=site_vertices[:, 1],
            z=site_vertices[:, 2],
            i=site_faces[:, 0],
            j=site_faces[:, 1],
            k=site_faces[:, 2],
            color="lightgray",
            opacity=0.3,
            name="Site",
            showscale=False,
        )
    )

    # Plot each variant
    for v in variants:
        verts = v.vertices
        faces = v.faces
        fig.add_trace(
            go.Mesh3d(
                x=verts[:, 0],
                y=verts[:, 1],
                z=verts[:, 2],
                i=faces[:, 0],
                j=faces[:, 1],
                k=faces[:, 2],
                color=v.color,
                opacity=0.8,
                name=v.name,
                showscale=False,
            )
        )

    minx, miny, maxx, maxy = site_polygon.bounds
    max_height = max(v.height for v in variants) * 1.1

    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=show_axes),
            yaxis=dict(visible=show_axes),
            zaxis=dict(visible=show_axes, range=[0, max_height]),
            aspectmode="data",
        ),
        width=900,
        height=700,
        title="Urban Massing Generator — 3D Envelopes",
    )

    fig.show()

plot_massing_3d(site_polygon, variants)

In [ ]:
import json
import csv

variant_summaries = [v.summary() for v in variants]

# JSON export
with open("massing_variants_summary.json", "w") as f:
    json.dump(variant_summaries, f, indent=2)

# CSV export
csv_fields = list(variant_summaries[0].keys())
with open("massing_variants_summary.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=csv_fields)
    writer.writeheader()
    for row in variant_summaries:
        writer.writerow(row)

print("Exported massing_variants_summary.json and massing_variants_summary.csv")

Exported massing_variants_summary.json and massing_variants_summary.csv


In [ ]:
def export_variant_to_obj(variant: MassingVariant, filename: str):
    """
    Very simple OBJ writer: one object, triangular faces only.
    """
    verts = variant.vertices
    faces = variant.faces

    with open(filename, "w") as f:
        f.write(f"# OBJ export for {variant.name}\n")
        for v in verts:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")
        for face in faces:
            # OBJ is 1-based indexing
            f.write(f"f {face[0]+1} {face[1]+1} {face[2]+1}\n")

    print(f"Exported {variant.name} to {filename}")

# Example: export the max block
export_variant_to_obj(variant_A, "max_block.obj")

Exported Max Block to max_block.obj
